In [2]:
import os
import numpy as np
import cv2

data_path = "./../data/raw_data/"

In [ ]:
#Run to segment regular black and white CAPTCHAs
def segmentation_preprocess():
    for image_name in os.listdir(data_path):
        img_path = data_path + image_name
        img = cv2.imread(img_path)
        _, thresholded = cv2.threshold(img, 240, 255, cv2.THRESH_BINARY_INV)
        to_gray = cv2.cvtColor(thresholded, cv2.COLOR_BGR2GRAY)
        ctrs, _ = cv2.findContours(to_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        ctrs = np.asarray(ctrs, dtype=object)
        my_rects = np.asarray([cv2.boundingRect(ctr) for ctr in ctrs])

        while len(my_rects) != 4:
            big_ctr_ind = np.argmax(my_rects[:, 2])
            big_ctr = my_rects[big_ctr_ind]
            split_width = big_ctr[2]//2
            sub_rect_one = np.asarray([big_ctr[0], big_ctr[1], split_width, 0])
            sub_rect_two = np.asarray([big_ctr[0] + split_width, big_ctr[1], big_ctr[2] - split_width, 0])
            my_rects = np.concatenate((my_rects[:big_ctr_ind], np.asarray([sub_rect_one, sub_rect_two]), my_rects[big_ctr_ind + 1:]))
        
        assert(len(my_rects) == 4)
        my_rects = np.sort(my_rects, axis=0)
        code = image_name.split(".")[0]
        for i, rect in enumerate(my_rects):
            new_image = to_gray[:,rect[0]-1:rect[0]+rect[2]+1] #Add one pixel of noise from original image
            total_pad = 24 - new_image.shape[1]
            left_pad  = total_pad // 2
            right_pad = total_pad - left_pad
            new_image = np.pad(new_image, pad_width=((0,0), (left_pad, right_pad)), mode="constant")
            cv2.imwrite("../data/segmented_data/" + code + "_" + str(i) + "_" + code[i] + ".png", new_image)

In [2]:
#Barcode the CAPTCHAs
def barcode_preprocess():
    for image_name in os.listdir(data_path):
        img_path = data_path + image_name
        img = cv2.imread(img_path)
        _, thresholded = cv2.threshold(img, 240, 255, cv2.THRESH_BINARY_INV)
        to_gray = cv2.cvtColor(thresholded, cv2.COLOR_BGR2GRAY)
        
        code = image_name.split(".")[0]
        for i, letter in enumerate(code):
            new_img = to_gray
            if i > 0:
                new_img = np.concatenate((np.ones((new_img.shape[0], i))*255, new_img), axis=1)
                
            new_img = np.concatenate((np.zeros((new_img.shape[0], 1)), new_img), axis=1)
            if i < 3:
                new_img = np.concatenate((np.ones((new_img.shape[0], 3-i))*255, new_img), axis=1)
            cv2.imwrite("./../data/barcoded_data/" + code + "_" + str(i) + "_" + letter + ".png", new_img)

In [ ]:
# Preprocess data for OCR
def ocr_preprocess():
    for image_name in os.listdir(data_path):
        img_path = data_path + image_name
        img = cv2.imread(img_path)
        _, thresholded = cv2.threshold(img, 240, 255, cv2.THRESH_BINARY_INV)
        to_gray = cv2.cvtColor(thresholded, cv2.COLOR_BGR2GRAY)
        cv2.imwrite("../data/ocr_data/" + image_name, to_gray)